# Geometry optimization

We use DFTK and the [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
package to find the minimal-energy bond length of the $H_2$ molecule.
First we set up an appropriate `DFTKCalculator` (see AtomsCalculators integration),
for which we use the LDA model just like in the Tutorial for silicon
in combination with a pseudodojo pseudopotential (see Pseudopotentials).

In [1]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
calc = DFTKCalculator(;
    model_kwargs = (; functionals=LDA(), pseudopotentials),  # model_DFT keyword arguments
    basis_kwargs = (; kgrid=[1, 1, 1], Ecut=10)  # PlaneWaveBasis keyword arguments
)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), pseudopotentials=PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf"), Ecut=10, kgrid=[1, 1, 1])

Next we set up an initial hydrogen molecule within a box of vacuum.
We use the parameters of the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/)
and DFTK's AtomsBase integration to setup the hydrogen molecule.
We employ a simulation box of 10 bohr times 10 bohr times 10 bohr and a
pseudodojo pseudopotential.

In [2]:
using LinearAlgebra
using Unitful
using UnitfulAtomic

r0 = 1.4   # Initial bond length in Bohr
a  = 10.0  # Box size in Bohr

cell_vectors = [[a, 0, 0]u"bohr", [0, a, 0]u"bohr", [0, 0, a]u"bohr"]
h2_crude = periodic_system([:H => [0, 0, 0.]u"bohr",
                            :H => [r0, 0, 0]u"bohr"],
                           cell_vectors)

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [       0,        0,        0]u"a₀")
    Atom(H,  [     1.4,        0,        0]u"a₀")


Finally we call `minimize_energy!` to start the geometry optimisation.
We use `verbosity=2` to get some insight into the minimisation.
With `verbosity=1` only a summarising table would be printed and with
`verbosity=0` (default) the minimisation would be quiet.

In [3]:
using GeometryOptimization
results = minimize_energy!(h2_crude, calc; tol_forces=2e-6, verbosity=2)
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime 
---   ---------------   ---------   ---------   ----   ----   ------
  1   -1.110666238630                   -0.83   0.80    8.0    3.15s
  2   -1.117199727575       -2.18       -1.92   0.80    1.0    1.18s
  3   -1.117250304515       -4.30       -2.70   0.80    1.0   20.8ms
  4   -1.117251117555       -6.09       -3.38   0.80    1.0   63.8ms
  5   -1.117251186044       -7.16       -4.15   0.80    1.0   21.3ms
  6   -1.117251190067       -8.40       -4.90   0.80    2.0   23.2ms
  7   -1.117251190139      -10.14       -5.43   0.80    1.0   21.5ms
  8   -1.117251190143      -11.38       -6.21   0.80    1.0   31.5ms
  9   -1.117251190143      -13.07       -6.52   0.80    1.0   22.2ms
 10   -1.117251190143      -13.56       -6.91   0.80    1.0   22.4ms
 11   -1.117251190143      -14.48       -7.62   0.80    1.0   25.6ms
 12   -1.117251190143      -14.65       -8.48   0.80    2.0   24.8ms
n     Energy            log10(ΔE) 

Structure after optimisation (note that the atom has wrapped around)

In [4]:
results.system

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [-0.0431829, -1.15151e-17, 1.81123e-17]u"a₀")
    Atom(H,  [ 1.44318, 3.79165e-17, -7.46796e-18]u"a₀")


Compute final bond length:

In [5]:
rmin = norm(position(results.system[1]) - position(results.system[2]))
println("Optimal bond length: ", rmin)

Optimal bond length: 1.486365869878153 a₀


Our results (1.486 Bohr) agrees with the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/).

> **Cell optimisations**
>
> Recent versions of [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
> support cell shape optimisations as well by passing `variablecell=true` to `minimize_energy!`.
> See the
> [GeometryOptimization documentation](https://juliamolsim.github.io/GeometryOptimization.jl/stable/examples/variablecell/)
> for an example.